In [1]:
! pip install faiss-cpu

     |████████████████████████████████| 8.4 MB 5.1 MB/s 


In [2]:
! pip install transformers

     |████████████████████████████████| 2.6 MB 4.9 MB/s 
     |████████████████████████████████| 636 kB 51.1 MB/s 
     |████████████████████████████████| 3.3 MB 42.6 MB/s 
     |████████████████████████████████| 895 kB 65.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
! pip install sentence-transformers

     |████████████████████████████████| 85 kB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 8.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126709 sha256=6afebce53ae3f8294c6419432982e5cdb56e30c44c9446900e55cf5444b536c5
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers


In [4]:
from transformers import AutoModel, AutoTokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
import json

In [7]:
file_handle = open("/content/drive/MyDrive/sentences.json")

In [8]:
sentences_data = json.load(file_handle)

In [9]:
sentences_data

['A pandemic is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people.',
 'The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century.',
 'Current pandemics include COVID-19 (SARS-CoV-2) and HIV/AIDS.',
 'As of 2018, approximately 37.9 million people are infected with HIV globally.',
 'Cholera is an infection of the small intestine by some strains of the bacterium Vibrio cholerae.',
 'Classic cholera symptom is large amounts of watery diarrhea that lasts a few days. Vomiting and muscle cramps may also occur. Diarrhea can be so severe that it leads within hours to severe dehydration and electrolyte imbalance.',
 'The COVID-19 pandemic, also known as the coronavirus pandemic, is an ongoing pandemic of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome 

In [10]:
file_handle.close()

In [11]:
def encoder(document):

  return model(**tokenizer(document,return_tensors='pt'))[0].detach().squeeze()
  

In [12]:
documents_embeddings = []

for document in sentences_data:

  documents_embeddings.append(encoder(document))

In [13]:
for embedding in documents_embeddings:

  print(embedding.size())

torch.Size([35, 768])
torch.Size([37, 768])
torch.Size([25, 768])
torch.Size([18, 768])
torch.Size([24, 768])
torch.Size([55, 768])
torch.Size([57, 768])
torch.Size([24, 768])
torch.Size([27, 768])
torch.Size([35, 768])
torch.Size([43, 768])


In [14]:
import torch

In [15]:
docs2vecs = []

for embedding in documents_embeddings:

  docs2vecs.append(torch.mean(embedding,dim=0))

In [16]:
import faiss

import numpy as np

In [17]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))

In [18]:
index.add_with_ids(np.array([doc_vec.numpy() for doc_vec in docs2vecs]),np.arange(0,len(sentences_data)))

In [19]:
index

<faiss.swigfaiss.IndexIDMap; proxy of <Swig Object of type 'faiss::IndexIDMapTemplate< faiss::Index > *' at 0x7f115ad1dd50> >

In [20]:
def search(query,k):

  encoded_query = encoder(query)

  encoded_query = torch.mean(encoded_query,dim=0)

  encoded_query = encoded_query.unsqueeze(dim=0).numpy()

  k_nn_docs = index.search(encoded_query,k)

  semantic_similarity_scores = k_nn_docs[0][0]

  results = [sentences_data[idx] for idx in k_nn_docs[1][0]]

  return list(zip(results,semantic_similarity_scores))

In [21]:
query_file_handle = open("/content/drive/MyDrive/questions.json")

In [22]:
queries = json.load(query_file_handle)

In [23]:
queries

['How many people have died during Black Death?',
 'Which diseases can be transmitted by animals?',
 'Connection between climate change and a likelihood of a pandemic',
 'What is an example of a latent virus',
 'Viruses in nanotechnology',
 'Giant viruses classification',
 'What are the notable pandemic prevention organizations?',
 'How many leprosy outbreaks are known to happen?',
 'What are the geographic areas with the highest transmission of malaria?',
 'How to prevent the spread of viral infections?']

In [33]:
queries[8]

'What are the geographic areas with the highest transmission of malaria?'

In [34]:
search(queries[8],1)

[('A pandemic is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people.',
  58.68607)]